## License 

Copyright 2019 Patrick Hall, Navdeep Gill, and the H2O.ai team

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

**DISCLAIMER:** This notebook is not legal compliance advice.

# Single Decision Tree Model, Shapley Feature Importance, and LIME Reason Codes

#### Python imports
In general, NumPy and Pandas will be used for data manipulation purposes and H2o and scikit-learn will be used for modeling tasks.

In [ ]:
# sklearn for single DT
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator # for LIME

import numpy as np   # array, vector, matrix calculations
import pandas as pd  # DataFrame handling

pd.options.display.max_columns = 999 # enable display of all columns in notebook

import shap # Python Shapley value package

# system packages for calling external graphviz processes
import os
import re
import subprocess

import operator # for sorting dictionaries

# in-notebook display
from IPython.display import Image
from IPython.display import display
%matplotlib inline

## Load, explore, and prepare UCI credit card default data

UCI credit card default data: https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

The UCI credit card default data contains demographic and payment information about credit card customers in Taiwan in the year 2005. The data set contains 23 input variables: 

* **`LIMIT_BAL`**: Amount of given credit (NT dollar)
* **`SEX`**: 1 = male; 2 = female
* **`EDUCATION`**: 1 = graduate school; 2 = university; 3 = high school; 4 = others 
* **`MARRIAGE`**: 1 = married; 2 = single; 3 = others
* **`AGE`**: Age in years 
* **`PAY_0`, `PAY_2` - `PAY_6`**: History of past payment; `PAY_0` = the repayment status in September, 2005; `PAY_2` = the repayment status in August, 2005; ...; `PAY_6` = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; ...; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
* **`BILL_AMT1` - `BILL_AMT6`**: Amount of bill statement (NT dollar). `BILL_AMNT1` = amount of bill statement in September, 2005; `BILL_AMT2` = amount of bill statement in August, 2005; ...; `BILL_AMT6` = amount of bill statement in April, 2005. 
* **`PAY_AMT1` - `PAY_AMT6`**: Amount of previous payment (NT dollar). `PAY_AMT1` = amount paid in September, 2005; `PAY_AMT2` = amount paid in August, 2005; ...; `PAY_AMT6` = amount paid in April, 2005. 

These input variables are used to predict the target variable, whether or not a customer defaulted on their credit card bill in late 2005.

#### Import data and clean

In [ ]:
# import XLS file
path = 'default_of_credit_card_clients.xls'
data = pd.read_excel(path,
                     skiprows=1)

# remove spaces from target column name 
data = data.rename(columns={'default payment next month': 'DEFAULT_NEXT_MONTH'}) 

#### Assign modeling roles
The shorthand name y is assigned to the prediction target. X is assigned to all other input variables in the credit card default data except the row indentifier, ID, and the demographic variables are dropped from the analysis.

In [ ]:
# assign target and inputs for decision tree
y = 'DEFAULT_NEXT_MONTH'
X = [name for name in data.columns if name not in [y, 'ID', 'AGE', 'EDUCATION', 'SEX', 'MARRIAGE', 'LIMIT_BAL']]
print('y =', y)
print('X =', X)

#### Display descriptive statistics

In [ ]:
data.describe()

#### Split data into training and test sets for early stopping

In [ ]:
np.random.seed(12345) # set random seed for reproducibility
split_ratio = 0.7     # 70%/30% train/test split

# execute split
split = np.random.rand(len(data)) < split_ratio
train = data[split]
test = data[~split]

# summarize split
print('Train data rows = %d, columns = %d' % (train.shape[0], train.shape[1]))
print('Test data rows = %d, columns = %d' % (test.shape[0], test.shape[1]))

## Train Single Decision Tree

#### Train decision tree classifier
Several different depths were assesed on validation data and these hyperparameters performed well enough. 

In [ ]:
clf = DecisionTreeClassifier(random_state=12345, max_depth=3)
clf.fit(train[X].values, train[y].values)
sklearn.tree.export_graphviz(clf, 'model.gv', feature_names=X, proportion=True)

#### Test AUC

In [ ]:
roc_auc_score(test[y].values, clf.predict_proba(test[X].values)[:, 1])

#### Create PNG from GraphViz dot file

In [ ]:
# construct call to generate PNG from 
# graphviz representation of the tree
png_args = str('dot -Tpng model.gv -o model.png')
png_args = png_args.split()

# call
print('Calling external process ...')
print(' '.join(png_args))
_ = subprocess.call(png_args)

## Visualize and Explain Single Tree

#### Display tree
This is the entire model, displayed as a directed graph.

In [ ]:
# display in-notebook
display(Image(('model.png')))

#### Display global Shapley variable importance
Shapley values are a locally-accurate and globally consistent variable importance metric. Instead of relying on traditional single-value variable importance measures, local Shapley values for each input will be calculated and aggregated below to get a more holistic and consisent measurement for the global importance of each input variable.

In [ ]:
explainer = shap.TreeExplainer(clf, test[X], model_output='probability', feature_dependence='independent')
shap_values = explainer.shap_values(test[X].values)
shap.summary_plot(shap_values[1], X, plot_type='bar', color='royalblue')

#### Find riskiest customer
The riskiest customer in this data set is selected from all the customers who have a probability of default of ~0.77.

In [ ]:
pd.options.mode.chained_assignment = None
yhat = 'p_DEFAULT_NEXT_MONTH'
test[yhat] = clf.predict_proba(test[X].values)[:, 1]
test.reset_index(inplace=True, drop=True)
test.sort_values(by=yhat, inplace=True)
id_ = test.iloc[-1, :].loc['ID']
idx = test.iloc[-1, :].name
pd.DataFrame(test.iloc[-1, :])

#### Display local Shapley values for the riskiest customer predictions

In [ ]:
s_df = pd.DataFrame(shap_values[1][idx, :], index=X)
s_df.sort_values(by=0, inplace=True)
_ = s_df[s_df[0] != 0].plot(kind='barh' , color='royalblue', legend=False)

The Shapley values combine the customer's data values, the rules of the displayed decision tree, and crucially, the rules of many similar perturbed decision trees to create locally accurate and globally consistent feature importance values for each customer in the dataset. Note that these values consider variables *not* on the customer's decision path. This is because Shapley values take other encodings of the signal in the dataset into account when creating local feature importance. Shapley values are one the few explanation methods that account for the so-called Rashomon effect in machine learning.

#### Check Shapley sum

It can be seen that the Shapley values are locally accurate. i.e. they sum to the model prediction for this customer. 

In [ ]:
s_df.sum()[0] + explainer.expected_value[1]

#### Approximate LIME trends
LIME uses linear models fit to a more complex machine learning function to explain that machine learning function.

In [ ]:
# select riskiest customers, those who are more than 1 month late on their most recent payment
local = test[(test['PAY_0'] > 1.5)] 

# variables with non-zero Global Shapley importance
local_X = s_df[s_df[0] != 0].index

h2o.init() # use h2o to train a linear model in the region of interest

# convert data into H2o frame
# treat categorical variables as such
local = h2o.H2OFrame(local)
local['PAY_0'] = local['PAY_0'].asfactor()
local['PAY_2'] = local['PAY_2'].asfactor()
local['PAY_5'] = local['PAY_5'].asfactor()
local['PAY_6'] = local['PAY_6'].asfactor()

# initialize
local_glm = H2OGeneralizedLinearEstimator(lambda_search=True, seed=12345)

# train 
local_glm.train(x=list(local_X), y=yhat, training_frame=local)

# coefs
print('\nLocal GLM Coefficients:')
for c_name, c_val in sorted(local_glm.coef().items(), key=operator.itemgetter(1)):
    if c_val != 0.0:
        print('%s %s' % (str(c_name + ':').ljust(25), c_val))
        
# r2
print('\nLocal GLM R-square:\n%.6f' % local_glm.r2())

Because the intercept of these LIMEs is logical and the LIMEs have a good R<sup>2</sup>, this information can be taken as a rough, linear description of the tree model's response surface in the region of interest, i.e. an average description for risky customers under the model.    

#### Assess LIME prediction accuracy for row of interest
It can also be seen that the LIMES have very good local accuracy for the riskiest customer.

In [ ]:
local_glm.predict(local[local['ID'] == id_])

#### Visually assess LIME model fit

In [ ]:
# ranked predictions plot
pred_frame = local_glm.predict(local).cbind(local)\
                       .as_data_frame()[['predict', yhat]]
pred_frame.columns = ['LIME Preds.', 'DT Preds.']
pred_frame.sort_values(by='DT Preds.', inplace=True)
pred_frame.reset_index(inplace=True, drop=True)
_ = pred_frame.plot(title='DT vs. LIME Ranked Predictions Plot')

This ranked predictions plot shows that the LIMEs follows the decision tree predictions for the most part and provides more evidence that the LIMES have decent fidelity to the decision tree model.

**Conclusion**: Pairing a directly interpretable model and Shapley values enabled visualization of an entire predictive model and accurate local feature importance values for any individual in the model. Combining Shapley values and LIME allows for comparison of the average behavior of customers in a region of interest and anyone single customer in the dataset.